In [1]:
#Use conda env hic_analysis_plots_conda_env.yml to create conda env for this script
#This is an example script of calculating the intermingling metric (IM) for real and permuted walks
#It is run from within the 'scripts' subdirectory, using following directory structure:
#Analysis_Dir
#├── data
#    ├── permutations
#├── alignments
#├── figures
#├── scripts
#├── lsf_jobs

In [2]:
import bioframe
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
import random
import seaborn as sns
import scipy
import pickle
from numpy import diff

from pandas import read_csv
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

%matplotlib inline

In [3]:
conditions = [
    'Ctrl_R1',
    'IAA_R1',
    'ICRF_R1',
    'IAA_ICRF_R1',
    'Ctrl_R2',
    'IAA_R2',
    'ICRF_R2',
    'IAA_ICRF_R2',
    'Ctrl_R3',
    'IAA_R3', 
    'ICRF_R3',
    'IAA_ICRF_R3'
]

long_names = {
    'Ctrl_R1' : 'TI-MC3C-Ctrl-R1-4-40',
    'IAA_R1' : 'TI-MC3C-IAA-R1-4-40',
    'ICRF_R1' : 'TI-MC3C-ICRF-R1-4-40',
    'IAA_ICRF_R1' : 'TI-MC3C-IAA-ICRF-R1-4-40',
    'Ctrl_R2' : 'TI-MC3C-Dpn-Ctrl-R3-5-06',
    'IAA_R2' : 'TI-MC3C-Dpn-IAA-R3-5-06',
    'ICRF_R2' : 'TI-MC3C-Dpn-ICRF-R3-5-06',
    'IAA_ICRF_R2' : 'TI-MC3C-Dpn-IAA-ICRF-R3-5-06',
    'Ctrl_R3' : 'TI-MC3C-Dpn-Ctrl-R4-5-13',
    'IAA_R3' : 'TI-MC3C-Dpn-IAA-R4-5-13', 
    'ICRF_R3' : 'TI-MC3C-Dpn-ICRF-R4-5-13',
    'IAA_ICRF_R3' : 'TI-MC3C-Dpn-IAA-ICRF-R4-5-13'
}


In [4]:
sampleColors = {
    'Ctrl_R1' : '#17BECF',
    'IAA_R1' : '#574D68',
    'ICRF_R1' : '#D62728',
    'IAA_ICRF_R1' : '#C6A15B',
    'Ctrl_R2' : '#17BECF',
    'IAA_R2' : '#574D68',
    'ICRF_R2' : '#D62728',
    'IAA_ICRF_R2' : '#C6A15B',
    'Ctrl_R3' : '#17BECF',
    'IAA_R3' : '#574D68',
    'ICRF_R3' : '#D62728',
    'IAA_ICRF_R3' : '#C6A15B'
}
    
sampleLineStyles = {
    'Ctrl_R1' : '-',
    'IAA_R1' : '-',
    'ICRF_R1' : '-',
    'IAA_ICRF_R1' : '-',
    'Ctrl_R2' : '--',
    'IAA_R2' : '--',
    'ICRF_R2' : '--',
    'IAA_ICRF_R2' : '--',
    'Ctrl_R3' : ':',
    'IAA_R3' : ':',
    'ICRF_R3' : ':',
    'IAA_ICRF_R3' : ':'
}

samplePlotNames = {
    'Ctrl_R1' : 'Ctrl_R1',
    'IAA_R1' : 'IAA_R1',
    'ICRF_R1' : 'ICRF_R1',
    'IAA_ICRF_R1' : 'IAA_ICRF_R1',
    'Ctrl_R2' : 'Ctrl_R2',
    'IAA_R2' : 'IAA_R2',
    'ICRF_R2' : 'ICRF_R2',
    'IAA_ICRF_R2' : 'IAA_ICRF_R2',
    'Ctrl_R3' : 'Ctrl_R3',
    'IAA_R3' : 'IAA_R3',
    'ICRF_R3' : 'ICRF_R3',
    'IAA_ICRF_R3' : 'IAA_ICRF_R3'
}

In [5]:
ctrlconds = [
    'Ctrl_R1',
    'Ctrl_R1',
    'Ctrl_R1',
    'Ctrl_R2',
    'Ctrl_R2',
    'Ctrl_R2',
    'Ctrl_R3',
    'Ctrl_R3',
    'Ctrl_R3'
]

treatconds = [
    'IAA_R1',
    'ICRF_R1',
    'IAA_ICRF_R1',
    'IAA_R2',
    'ICRF_R2',
    'IAA_ICRF_R2',
    'IAA_R3',
    'ICRF_R3',
    'IAA_ICRF_R3'
]

In [6]:
repdict = {
    'Ctrl_R1' : 'R1',
    'IAA_R1' : 'R1',
    'ICRF_R1' : 'R1',
    'IAA_ICRF_R1' : 'R1',
    'Ctrl_R2' : 'R2',
    'IAA_R2' : 'R2',
    'ICRF_R2' : 'R2',    
    'IAA_ICRF_R2' : 'R2',
    'Ctrl_R3' : 'R3',
    'IAA_R3' : 'R3',
    'ICRF_R3' : 'R3',    
    'IAA_ICRF_R3' : 'R3'
}

labeldict = {
    'Ctrl_R1' : 'Control',
    'IAA_R1' : 'IAA',
    'ICRF_R1' : 'ICRF-193',
    'IAA_ICRF_R1' : 'IAA + ICRF-193',
    'Ctrl_R2' : 'Control',
    'IAA_R2': 'IAA',
    'ICRF_R2' : 'ICRF-193',  
    'IAA_ICRF_R2' : 'IAA + ICRF-193',
    'Ctrl_R3' : 'Control',
    'IAA_R3': 'IAA',
    'ICRF_R3' : 'ICRF-193',  
    'IAA_ICRF_R3' : 'IAA + ICRF-193'
}

labelPlotColors = {
    'Control' : '#17BECF',
    'IAA' : '#574D68',
    'ICRF-193' : '#D62728',
    'IAA + ICRF-193' : '#C6A15B',
}


In [7]:
#For each of the above, can separate by A vs B compartments, or by number of chromosomes visited
comp_types = ['A', 'B', 'AB']

In [8]:
outDataDir = '..'

In [9]:
#Read in pickled files

f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_real_walks_with_summary_firstx_length_fractions_dict_100kbEigs.pkl', 'rb')
real_walks_with_summary_firstx = pickle.load(f)
f.close()

f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_real_walks_filtered_summarized_firstx_length_dict_100kbEigs.pkl', 'rb')
real_walks_summarized_firstx = pickle.load(f)
f.close()

In [10]:
#Calculate intermingling metric for real walks, first 6 steps, with QC filters

In [11]:
#Parameter sweep of window size, keep 1Mb window size step size for all. 

In [12]:
#Plotting entanglement for largest step - sliding window analysis

In [13]:
Intermingling_Sliding_Window_Sweep = {}

start_dist = 0 #start of first window
end_dist = 5e7 #end of last window

for window_size in [4e6, 6e6, 8e6, 1e7, 1.2e7, 1.4e7, 1.6e7]:
    window_step = 1e6
    
    Intermingling_Sliding_Window = pd.DataFrame()

    #set up windows
    for i, start_bp in enumerate(range(int(start_dist), int(end_dist-window_size), int(window_step))):
        end_bp = start_bp + int(window_size)

        #iterate through conditions for each window, all compartments
        for cond in conditions:
            df = real_walks_with_summary_firstx[cond]['length_6']
            df['Query_Fragment_Length'] = df['Query_End'] - df['Query_Start']
            grouped_walks = df.groupby('Query_Name')
            walks_min_mapq = grouped_walks.agg({'Mapping_Quality' : 'min'})
            good_walks_mapq = walks_min_mapq[walks_min_mapq['Mapping_Quality'] > 59] #use this to filter for mapq
            walks_frac_map = grouped_walks.agg({'Match_Length' : 'sum',
                                        'Query_Fragment_Length' : 'sum',
                                        'Alignment_Length' : 'sum'
                                       })
            walks_high_frac_map = walks_frac_map[
                (walks_frac_map['Match_Length']/walks_frac_map['Query_Fragment_Length']) > 0.8] #use this to filter for fraction mapped

            df2 = real_walks_summarized_firstx[cond]['length_6'].copy()
            df_cond = df2[
                (df2['Chrom_Number'] == 1) &
                (df2['Walk_Comp_Type'].isin(['A', 'B', 'AB'])) &
                (df2['Query_Name'].isin(good_walks_mapq.index)) &
                (df2['Query_Name'].isin(walks_high_frac_map.index)) &
                (df2['Near_Largest_Step_Either_Side_Frag_Num'] == 6) &
                (df2['Largest_Step'] >= start_bp) &
                (df2['Largest_Step'] < end_bp) &
                (df2['Max_NearOneLargestStepEnd_FracOfFragments'] == 5/6)
            ][['Inter_Largest_Step_Side_Step_Num']]
            df_cond['Condition'] = cond
            df_cond['Label'] = labeldict[cond]
            df_cond['Window_Midpoint'] = (start_bp + end_bp)//2
            df_cond['Walk_Comp_Type'] = 'All'
            df_cond['Replicate'] = repdict[cond]

            #add together into one dataframe
            Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_cond, ignore_index = True)

            for comp in comp_types:
                #iterate through comp types
                df_comp = df2[
                    (df2['Chrom_Number'] == 1) &
                    (df2['Walk_Comp_Type'] == comp) &
                    (df2['Query_Name'].isin(good_walks_mapq.index)) &
                    (df2['Query_Name'].isin(walks_high_frac_map.index)) &
                    (df2['Near_Largest_Step_Either_Side_Frag_Num'] == 6) &
                    (df2['Largest_Step'] >= start_bp) &
                    (df2['Largest_Step'] < end_bp) &
                    (df2['Max_NearOneLargestStepEnd_FracOfFragments'] == 5/6)
                ][['Inter_Largest_Step_Side_Step_Num']]
                df_comp['Condition'] = cond
                df_comp['Label'] = labeldict[cond]
                df_comp['Window_Midpoint'] = (start_bp + end_bp)//2
                df_comp['Walk_Comp_Type'] = comp
                df_comp['Replicate'] = repdict[cond]

                #add together into one dataframe
                Intermingling_Sliding_Window = Intermingling_Sliding_Window.append(df_comp, ignore_index = True)
            
    Intermingling_Sliding_Window_Sweep[f'{window_size}'] = Intermingling_Sliding_Window

In [14]:
#save intermingling results as pickle file for later use since takes a long time to run
f = open(f'{outDataDir}/data/Rad21ICRF_R1R2R3_InterminglingSlideingWindowSweep.pkl', 'wb')
pickle.dump(Intermingling_Sliding_Window_Sweep, f)
f.close()

In [15]:
#permutations - calculate intermingling metric

In [16]:
#submit permutations for sliding window sweep intermingling analysis

In [20]:
iterations = range(100)
window_sizes = [4e6, 6e6, 8e6, 1e7, 1.2e7, 1.4e7, 1.6e7]

In [21]:
for i in iterations:
    for window in window_sizes:
        !bsub -q short -W 00:20 -e ../lsf_jobs/LSB_%J.err -o ../lsf_jobs/LSB_%J.log \
            -n 1 -R span[hosts=1] -R select[ib] -R rusage[mem=3000] -R select[rh=8] \
            "python Rad21ICRF_Intermingling_Permutations.py $i $outDataDir $window"

Job <4938179> is submitted to queue <short>.
Job <4938180> is submitted to queue <short>.
Job <4938181> is submitted to queue <short>.
Job <4938182> is submitted to queue <short>.
Job <4938183> is submitted to queue <short>.
Job <4938184> is submitted to queue <short>.
Job <4938185> is submitted to queue <short>.
Job <4938186> is submitted to queue <short>.
Job <4938187> is submitted to queue <short>.
Job <4938189> is submitted to queue <short>.
Job <4938190> is submitted to queue <short>.
Job <4938191> is submitted to queue <short>.
Job <4938192> is submitted to queue <short>.
Job <4938193> is submitted to queue <short>.
Job <4938194> is submitted to queue <short>.
Job <4938195> is submitted to queue <short>.
Job <4938196> is submitted to queue <short>.
Job <4938197> is submitted to queue <short>.
Job <4938198> is submitted to queue <short>.
Job <4938199> is submitted to queue <short>.
Job <4938200> is submitted to queue <short>.
Job <4938201> is submitted to queue <short>.
Job <49382

Job <4938367> is submitted to queue <short>.
Job <4938368> is submitted to queue <short>.
Job <4938369> is submitted to queue <short>.
Job <4938370> is submitted to queue <short>.
Job <4938371> is submitted to queue <short>.
Job <4938372> is submitted to queue <short>.
Job <4938373> is submitted to queue <short>.
Job <4938374> is submitted to queue <short>.
Job <4938375> is submitted to queue <short>.
Job <4938376> is submitted to queue <short>.
Job <4938377> is submitted to queue <short>.
Job <4938378> is submitted to queue <short>.
Job <4938379> is submitted to queue <short>.
Job <4938380> is submitted to queue <short>.
Job <4938381> is submitted to queue <short>.
Job <4938382> is submitted to queue <short>.
Job <4938383> is submitted to queue <short>.
Job <4938384> is submitted to queue <short>.
Job <4938385> is submitted to queue <short>.
Job <4938386> is submitted to queue <short>.
Job <4938387> is submitted to queue <short>.
Job <4938388> is submitted to queue <short>.
Job <49383

Job <4938552> is submitted to queue <short>.
Job <4938553> is submitted to queue <short>.
Job <4938554> is submitted to queue <short>.
Job <4938555> is submitted to queue <short>.
Job <4938556> is submitted to queue <short>.
Job <4938557> is submitted to queue <short>.
Job <4938558> is submitted to queue <short>.
Job <4938559> is submitted to queue <short>.
Job <4938560> is submitted to queue <short>.
Job <4938561> is submitted to queue <short>.
Job <4938562> is submitted to queue <short>.
Job <4938563> is submitted to queue <short>.
Job <4938564> is submitted to queue <short>.
Job <4938565> is submitted to queue <short>.
Job <4938566> is submitted to queue <short>.
Job <4938567> is submitted to queue <short>.
Job <4938568> is submitted to queue <short>.
Job <4938569> is submitted to queue <short>.
Job <4938570> is submitted to queue <short>.
Job <4938571> is submitted to queue <short>.
Job <4938572> is submitted to queue <short>.
Job <4938573> is submitted to queue <short>.
Job <49385

Job <4938737> is submitted to queue <short>.
Job <4938738> is submitted to queue <short>.
Job <4938739> is submitted to queue <short>.
Job <4938740> is submitted to queue <short>.
Job <4938741> is submitted to queue <short>.
Job <4938742> is submitted to queue <short>.
Job <4938743> is submitted to queue <short>.
Job <4938744> is submitted to queue <short>.
Job <4938745> is submitted to queue <short>.
Job <4938746> is submitted to queue <short>.
Job <4938747> is submitted to queue <short>.
Job <4938748> is submitted to queue <short>.
Job <4938749> is submitted to queue <short>.
Job <4938750> is submitted to queue <short>.
Job <4938751> is submitted to queue <short>.
Job <4938752> is submitted to queue <short>.
Job <4938753> is submitted to queue <short>.
Job <4938754> is submitted to queue <short>.
Job <4938755> is submitted to queue <short>.
Job <4938756> is submitted to queue <short>.
Job <4938758> is submitted to queue <short>.
Job <4938760> is submitted to queue <short>.
Job <49387